In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

In [3]:
#数据载入---有其他方法
mnist = input_data.read_data_sets("MNIST_data",one_hot=True) #注意输出标签是one_hot类型

#按批次进行计算
#设置每个批次大小
batch_size = 100
#获取批次数量，注意：我们训练时使用的是mnist.train训练集数据
n_batch = mnist.train.num_examples // batch_size #注意：//也是除法，返回整数

#参数概要https://www.cnblogs.com/lyc-seu/p/8647792.html
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var) #求均值
        tf.summary.scalar("mean",mean) #用来显示标量信息
        tf.summary.scalar("stddev",tf.sqrt(tf.reduce_mean(tf.square(var-mean))))
        tf.summary.scalar("max",tf.reduce_max(var))
        tf.summary.scalar("min",tf.reduce_min(var))
        tf.summary.histogram("histogram",var) #直方图

#设置命名空间--输入层
with tf.name_scope("input"): #命名空间的使用：https://www.jianshu.com/p/635d95b34e14
    #开始定义占位符号
    x = tf.placeholder(tf.float32,[None,784],name='x-input') #由于每张图片都是28×28像素，所以每个样本的特征都是784
    y = tf.placeholder(tf.float32,[None,10],name='y-input') #是分10类

#设置命名空间--输出层
with tf.name_scope("layer"):
    #开始创建一个神经网络隐藏层L1含有10个激活单元
    with tf.name_scope("weight"):
        Weight_L1 = tf.Variable(tf.zeros([784,10]),name='w')
        variable_summaries(Weight_L1)
    with tf.name_scope("biases"):
        biases_L1 = tf.Variable(tf.zeros([1,10]),name='b') #对于偏执单元的权重，我们没有必要进行随机初始化
        variable_summaries(biases_L1)
    with tf.name_scope("wx_plus_b"):
        Wx_plus_b_L1 = tf.matmul(x,Weight_L1)+biases_L1
    with tf.name_scope("prediction"):
        y_pred = tf.nn.softmax(Wx_plus_b_L1)
        
with tf.name_scope("loss"):
    #定义代价函数
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=y_pred)) #这里我们使用交叉熵进行处理，使得加快训练速度
    tf.summary.scalar("loss",loss)
    
with tf.name_scope("train"):    
    #使用梯度下降法
    train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

init = tf.global_variables_initializer()

with tf.name_scope("accuracy"):
    with tf.name_scope("correct_prediction"):
        #获取结果存放在bool列表中
        correct_prediction = tf.equal(tf.arg_max(y,1),tf.arg_max(y_pred,1)) #注意：arg_max第二个参数是指定坐标信息，因为结果按行存储，所以1
    with tf.name_scope("accuracy"):
        #根据上面的bool列表，获取准确率
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32)) #tf.cast格式转换后，求解平均值
        tf.summary.scalar("accuracy",accuracy)

merged = tf.summary.merge_all() #合并所有的summarymerge_all 可以将所有summary全部保存到磁盘，以便tensorboard显示。如果没有特殊要求，一般用这一句就可以显示训练时的各种信息了。

#开启会话
with tf.Session() as sess:
    sess.run(init)
    writer = tf.summary.FileWriter('logs/',sess.graph)  #Summary用法https://www.cnblogs.com/lyc-seu/p/8647792.html
    for iter_cnt in range(51): #整体数据迭代
        for batch_cnt in range(n_batch): #小批次数据分批次迭代
            batch_xs,batch_ys = mnist.train.next_batch(batch_size) #依照batch_size获取下一批次数据
            summary,_ = sess.run([merged,train_step],feed_dict={x:batch_xs,y:batch_ys}) #统计和训练一块进行，获取merged返回值

        writer.add_summary(summary,iter_cnt) #将summary和运行周期存放到文件中
        #获取每一次整体数据迭代后的准确率
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels}) #注意：预测时，我们也要传入数据，并且传入的是测试集数据
        print("Iter: %d, Testing accuracy:%f"%(iter_cnt,acc))


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Instructions for updating:
Use `tf.math.argmax` instead
Iter: 0, Testing accuracy:0.825700
Iter: 1, Testing ac